# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [34]:
import sys
sys.path.append(r"D:\01. Projecten\[130878] DTV vaarwegen\digitaltwin-waterway\dtv_backend")

import datetime

import logging
import geojson
import simpy
import time
import json
import shapely
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.berthing
import dtv_backend.simulate
import dtv_backend.postprocessing

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('config.geojson') as f:
    config = geojson.load(f)

### Dummy env

In [5]:
logger = logging.getLogger()

now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

# default no berth
with_berth = config.get("options", {}).get("with_berth", False)

# read the network from google for performance reasons
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
G = dtv_backend.fis.load_fis_network(url)
env.FG = G

## WIP

In [35]:

logger.info("Loading ports ⚓")
ports = []
for site in config["sites"]:
    port = dtv_backend.simple.Port(env, **site["properties"], **site)
    ports.append(port)

logger.info("Loading ships 🚢")
ships = []
for ship in config["fleet"]:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship["properties"])
    # the ship needs to know about the climate
    if "climate" in config:
        kwargs["climate"] = config["climate"]
    ship = dtv_backend.simple.Ship(env, **kwargs)
    ships.append(ship)

logger.info("Loadig operator 👩‍💼")
# Setup and start the simulation
operator = dtv_backend.simple.Operator(env, ships=ships, **config["operator"])
# The ships do work for the operator
for ship in ships:
    env.process(ship.work_for(operator, with_berth=with_berth))
# The opertor plans the work move everything from A to B
env.process(operator.plan(ports[0], ports[1]))

logger.info("Running simulation 👩‍💻")
# Run for n days
n_days_in_future = now + datetime.timedelta(days=60)

In [8]:
ports

In [9]:
ships

In [10]:
operator

In [14]:
config['climate']

{'discharge_lobith': 6000, 'discharge_st_pieter': 1300, 'sealevel': 0}

In [24]:
ship = ships[0]

In [33]:
config['fleet'][0]['properties']

{'CEMT-class': 'Va',
 'RWS-class': 'M8',
 'Vessel type': 'Motorvessel',
 'Description (English)': 'Large Rhine vessel',
 'Description (Dutch)': 'Groot Rijnschip',
 'Beam [m]': 11.4,
 'Beam 10% percentile [m]': 10.6,
 'Beam 50% percentile [m]': 11.45,
 'Beam 90% percentile [m]': 11.45,
 'Length [m]': 110,
 'Length 10% percentile [m]': 93,
 'Length 50% percentile [m]': 110,
 'Length 90% percentile [m]': 110,
 'Draught loaded [m]': 3.5,
 'Draught average [m]': 3.14,
 'Draught empty [m]': 1.4,
 'Height average [m]': 6.21,
 'Load Weight average [ton]': 2286,
 'Load weight maximum [ton]': 2689,
 'speed': 3,
 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRvKRniAxUXUWzmByw7CRFYD5fTqOtFTDVkw&usqp=CAU',
 'capacity': 2286,
 'name': 'Schip',
 'count': 4}

In [40]:
ship.cargo.level

2286

In [48]:
config['sites'][0]['properties']

{'Wkt': 'POINT (4.0874555158298396 51.9653093741551970)',
 'X': 4.08745551582984,
 'Y': 51.9653093741552,
 'n': '8861022',
 'name': '8861022',
 'cargoType': 'Dry Bulk',
 'capacity': 10000,
 'level': 10000,
 'loadingRate': 198,
 'loadingRateVariation': 102}

In [47]:
port = ports[0]
port.loading_rate

1

In [36]:
env.run(until=n_days_in_future.timestamp())

In [ ]:
dtv_backend.network.network_utilities.path2gdf(path, graph)

In [ ]:
ship = out['ships'][0]

In [ ]:
ship.cargo.capacity

In [ ]:
ship.logbook

In [ ]:
df_log = pd.DataFrame(out['operator'].logbook)

In [ ]:
df_log

In [ ]:
dtv_backend.postprocessing.log2gantt(df_log)